## 과제 수행자: 이세현

## 학번: 2025178869

# 02-2 스크래핑 사용하기

## 과제 :  ‘무게’까지 추출해 별도 함수‧데이터프레임 만들기

In [41]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_page_and_weight(isbn: str):
    """
    Yes24 상세 페이지에서
    ─ 쪽수            (예: '324쪽')
    ─ 무게(gram)      (예: '580g')
    를 동시에 추출해 (page_cnt, weight_g) 튜플로 반환한다.
    """
    search_url = f"http://www.yes24.com/Product/Search?domain=BOOK&query={isbn}"
    r = requests.get(search_url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")

    # 검색 결과 중 첫 번째 도서 링크
    anchor = soup.find("a", class_="gd_name")
    if anchor is None:                       # ISBN이 없거나 검색 실패 시
        return "", ""

    detail_url  = "http://www.yes24.com" + anchor["href"]
    r = requests.get(detail_url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")

    spec_trs = soup.select("#infoset_specific tr")
    for tr in spec_trs:
        if tr.find("th").get_text(strip=True) == "쪽수, 무게, 크기":
            raw = tr.find("td").get_text(" ", strip=True)
            tokens = re.split(r"[|,\s]+", raw)      # '324쪽 580g 152*225*30mm' → 토큰 분리

            page_cnt = next((t.replace("쪽", "") for t in tokens if "쪽" in t), "")
            weight_g = next((t.replace("g",  "") for t in tokens if "g"  in t), "")
            return page_cnt, weight_g
    return "", ""

# ─────────────────────────────────────────────────────────────
# 예시: 상위 10권 책 데이터프레임(top10_books)에 컬럼 추가
# ─────────────────────────────────────────────────────────────
page_weight_df = top10_books["isbn13"].apply(
    lambda x: pd.Series(get_page_and_weight(x), index=["page_cnt", "weight_g"])
)

top10_enriched = pd.concat([top10_books, page_weight_df], axis=1)
top10_enriched.head()


,no,ranking,bookname,authors,publisher,publication_year,isbn13,page_cnt,weight_g
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,344,496
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,300,358
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,228,296
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,340,412
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,264,388


## get_page_cnt()를 확장해 ‘쪽수+무게’ 두 값을 한꺼번에 뽑기

In [42]:
def get_page_cnt(isbn: str):
    """
    쪽수(page_cnt)와 무게(weight_g)를 동시에 반환하도록 확장한 버전.
    사용법은 동일하지만 결과가 (page_cnt, weight_g) 튜플이다.
    """
    search_url = f"http://www.yes24.com/Product/Search?domain=BOOK&query={isbn}"
    r = requests.get(search_url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")

    anchor = soup.find("a", class_="gd_name")
    if anchor is None:
        return "", ""

    detail_url = "http://www.yes24.com" + anchor["href"]
    r = requests.get(detail_url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")

    for tr in soup.select("#infoset_specific tr"):
        if tr.find("th").get_text(strip=True) == "쪽수, 무게, 크기":
            raw = tr.find("td").get_text(" ", strip=True)
            tokens = re.split(r"[|,\s]+", raw)
            page_cnt = next((t.replace("쪽","") for t in tokens if "쪽" in t), "")
            weight_g = next((t.replace("g","")  for t in tokens if "g"  in t), "")
            return page_cnt, weight_g
    return "", ""

# ─────────────────────────────────────────────────────────────
# 두 컬럼(page_cnt, weight_g)을 한 번에 추가
# ─────────────────────────────────────────────────────────────
top10_books[["page_cnt", "weight_g"]] = top10_books["isbn13"].apply(
    lambda isbn: pd.Series(get_page_cnt(isbn))
)

top10_books.head()


<ipython-input-42-d0dfc85b2c61>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top10_books[["page_cnt", "weight_g"]] = top10_books["isbn13"].apply(
<ipython-input-42-d0dfc85b2c61>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top10_books[["page_cnt", "weight_g"]] = top10_books["isbn13"].apply(


,no,ranking,bookname,authors,publisher,publication_year,isbn13,page_cnt,weight_g
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,344,496
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,300,358
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,228,296
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,340,412
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,264,388
